In [48]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()

In [49]:
model = YOLO('C:/Users/User/Desktop/428/Final/CSSE428 Project Evaluations-20240502T143528Z-001/CSSE428 Project Evaluations/best.pt')
prediction_results = model.predict(source='C:/Users/User/Desktop/428/Final/CSSE428 Project Evaluations-20240502T143528Z-001/CSSE428 Project Evaluations/test/images',save=True)




image 1/3 C:\Users\User\Desktop\428\Final\CSSE428 Project Evaluations-20240502T143528Z-001\CSSE428 Project Evaluations\test\images\22.png: 384x640 4 SignBoards, 115.7ms
image 2/3 C:\Users\User\Desktop\428\Final\CSSE428 Project Evaluations-20240502T143528Z-001\CSSE428 Project Evaluations\test\images\37.png: 384x640 2 SignBoards, 114.7ms
image 3/3 C:\Users\User\Desktop\428\Final\CSSE428 Project Evaluations-20240502T143528Z-001\CSSE428 Project Evaluations\test\images\43.png: 384x640 1 SignBoard, 100.0ms
Speed: 0.3ms preprocess, 110.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\User\Desktop\428\New folder\ultralytics\runs\detect\predict8


In [52]:
""" 
IoU Calculation

"""



import numpy as np
import pprint
import os

prediction_list = []

for prediction_result in prediction_results:
    boxes_xywhn = prediction_result.boxes.xywhn

    boxes_xywhn_numpy = boxes_xywhn.numpy()
    boxes_xywhn_lists = boxes_xywhn_numpy.tolist()

    prediction_list.append(boxes_xywhn_lists)



# Directory containing the label files
label_folder = './test/labels'

# Initialize an empty list to store lines from all files
ground_truths = []

# Iterate through each file in the directory
for filename in os.listdir(label_folder):
    if filename.endswith('.txt'):
        file_path = os.path.join(label_folder, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            lines = [line.strip().split() for line in lines]  # Remove leading/trailing whitespaces
            ground_truths.append(lines)


# Now all_lines contains a list of lists, where each inner list contains the lines of one file


for i in range(len(ground_truths)):
    for j in range(len(ground_truths[i])):
        ground_truths[i][j] = ground_truths[i][j][1:]
        for k in range(len(ground_truths[i][j])):
            ground_truths[i][j][k] = float(ground_truths[i][j][k])








def calculate_iou(box1, box2):
    """
    
    box1 and box2  in the format [x, y, w, h].
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    """

    Finding Bottom_Left and Top_Right coordinates of the prediction and ground_truth boxes

    """
    x1_bottom_left = x1 - w1/2
    x1_top_right = x1 + w1/2

    y1_bottom_left = y1 - h1/2
    y1_top_right = y1 + h1/2



    x2_bottom_left = x2 - w2/2
    x2_top_right = x2 + w2/2

    y2_bottom_left = y2 - h2/2
    y2_top_right = y2 + h2/2
    

    """

    Calculate coordinates of  top right and bottom left of intersection rectangle

    """

    x_intersection_top_right = min(x1_top_right, x2_top_right)
    y_intersection_top_right = min(y1_top_right, y2_top_right)
    x_intersection_bottom_left = max(x1_bottom_left, x2_bottom_left)
    y_intersection_bottom_left = max(y1_bottom_left, y2_bottom_left)
    
    if x1_top_right < x2_bottom_left or y1_top_right < y2_bottom_left:
        return 0.0  # No intersection
    if x2_top_right < x1_bottom_left or y2_top_right < y1_bottom_left:
        return 0.0  # No intersection
    
    # Calculate area of intersection rectangle
    intersection_area = abs(x_intersection_top_right - x_intersection_bottom_left) * abs(y_intersection_top_right - y_intersection_bottom_left)
    
    # Calculate area of each bounding box
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # Calculate IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    
    return iou

def compute_iou_for_lists(predictions, ground_truths):
    """
    Compute IoU for each pair of bounding boxes in predictions and ground_truths lists.
    """
    iou_list = []
    for pred_boxes, gt_boxes in zip(predictions, ground_truths):
        iou_row = []
        for pred_box in pred_boxes:
            max_iou = 0
            for gt_box in gt_boxes:
                iou = calculate_iou(pred_box, gt_box)
                if iou > max_iou:
                    max_iou = iou
            iou_row.append(max_iou)
        iou_list.append(iou_row)
    return iou_list



iou_matrix = compute_iou_for_lists(prediction_list, ground_truths)
#pprint.pp(iou_matrix)
for i in range(len(iou_matrix)):
    total = 0
    for j in range(len(iou_matrix[i])):
        total += iou_matrix[i][j]
    average_iou = total/len(iou_matrix[i])
    print(f"Average IoU for {i+1} number image is {average_iou}")




Average IoU for 1 number image is 0.6977896299107496
Average IoU for 2 number image is 0.6734934846793412
Average IoU for 3 number image is 0.9327370892204389
